In [1]:
from math import sin, cos, tan, radians, sqrt, pi ,e


# ALU helper
def to_rpn(expression):
    try:
        tokens = tokenize(expression)
        rpn = shunting_yard(tokens)

        recipe = []
        for token in rpn:
            if isinstance(token, (int, float)):
                recipe.append(("PUSH", token))
            else:
                recipe.append(("OPERATOR", token))

        return recipe

    except Exception as e:
        print(f"ERROR in function: to_rpn(expression) = {str(e)}" )

def get_precedence(op):
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2, 'u-': 3, '**': 4, 'sin': 5, 'cos': 5, 'tan': 5, 'sqrt': 5}
    return precedence[op] if op in precedence else 0

# new
def shunting_yard(expression_tokens):
    output = []
    operators = []

    for token in expression_tokens:
        if isinstance(token, float) or token in ['pi', 'e']:  # Treat pi and e as numbers
            output.append(token)

        # # [Handling for operators and parentheses]
        # print(f"shunting_yard() processing token: {token}")
        # # [Existing shunting yard logic]
        # print(f"Current output: {output}, Current operators: {operators}")

        if '.' in token or token.replace('-', '', 1).isdigit():  # Handle floats and negative numbers
            output.append(float(token))
        elif token in ['+', '-', '*', '/', '**', 'sin', 'cos', 'tan', 'sqrt', 'abs']:
            while operators and operators[-1] != '(' and get_precedence(token) <= get_precedence(operators[-1]):
                output.append(operators.pop())
            operators.append(token)
        elif token == 'u-':
            operators.append(token)
        elif token == '(':
            operators.append(token)
        elif token == ')':
            while operators and operators[-1] != '(':
                output.append(operators.pop())
            operators.pop()  # Pop the '('

    while operators:
        output.append(operators.pop())

    # Ensure the shunting yard algorithm checks for unbalanced parentheses
    open_parentheses = 0
    for token in expression_tokens:
        if token == '(':
            open_parentheses += 1
        elif token == ')':
            open_parentheses -= 1
        # [Rest of the logic]
    if open_parentheses != 0:
        raise ValueError("Unbalanced parentheses in expression")

    # # Prints
    # for token in expression_tokens:
    #     print(f"Processing token: {token}")

    #     # [Existing shunting yard logic]
    #     print(f"Current operator stack: {operators}")
    #     print(f"Current output queue: {output}")

    # print(f"Final RPN output: {output}")
    return output


def tokenize(expression):
    expression = expression.replace(' ', '')   # Remove spaces
    expression = expression.replace('\n', '')  # Remove newline
    expression = expression.replace('\t', '')  # Remove newline

    tokens = []
    i = 0
    while i < len(expression):
        # print(f"tokenize(), current character: {expression[i]}, Position: {i}")  # Diagnostic print

        if expression[i] == ' ':
            i += 1
            continue

        # Check for exponentiation operator '**'
        if expression[i] == '*' and i + 1 < len(expression) and expression[i + 1] == '*':
            tokens.append('**')
            i += 2
            continue  # Skip the next character to avoid double-counting '*'

        # Handling binary operators, excluding the case where they are followed by a unary minus
        if expression[i] in '+*/' and i + 1 < len(expression) and expression[i + 1] in '+*/' and expression[i + 1] != '-':
            raise ValueError(f"Invalid consecutive operators: {expression[i]}{expression[i + 1]} at position {i}")


        elif expression[i] == '-':  # Handling unary minus
            # if i == 0 or expression[i-1] in '+-*/(' or (i > 1 and expression[i-1] == ' ' and expression[i-2] in '+-*/('):
            if i == 0 or expression[i-1] in '+-*/(':
                tokens.append('u-')  # Unary minus
                # print(f"Appended unary minus: u-")
            else:
                  # Check for consecutive binary operators (invalid syntax)
                  if i + 1 < len(expression) and expression[i + 1] in '+*/':
                      raise ValueError(f"Invalid consecutive operators: {expression[i]}{expression[i + 1]} at position {i}")
                  tokens.append('-')  # Binary minus (subtraction)
            i += 1

        # elif expression[i] in '+*/' and i + 1 < len(expression) and expression[i + 1] in '+-*/':  # Add a new elif block here to check for consecutive binary operators
        #     raise ValueError(f"Invalid consecutive operators: {expression[i]}{expression[i + 1]} at position {i}")

        elif expression[i] == 'e':  # Check for 'e'
            tokens.append('e')  # Append the value of e
            i += 1
        elif i + 1 < len(expression) and expression[i:i+2] == 'pi':  # Check for 'pi'
            tokens.append('pi')  # Append the value of pi
            i += 2
        elif expression[i] == '*' and i + 1 < len(expression) and expression[i + 1] == '*':
            tokens.append('**')
            i += 2
        elif expression[i:i+3] == 'sin' or expression[i:i+3] == 'cos' or expression[i:i+3] == 'tan':
            tokens.append(expression[i:i+3])
            i += 3
        elif (expression[i] == '-' and i + 1 < len(expression) and (expression[i + 1].isdigit() or expression[i + 1] == '.')) or expression[i].isdigit() or expression[i] == '.':
            j = i + 1
            while j < len(expression) and (expression[j].isdigit() or expression[j] == '.'):
                j += 1
            tokens.append(expression[i:j])
            i = j
        elif expression[i:i+3] == 'abs':
            tokens.append('abs')
            i += 3
            continue
        elif expression[i:i+4] == 'sqrt':
            tokens.append('sqrt')
            i += 4

        else:
            if not expression[i].isdigit() and expression[i] not in '+-*/()':
                print(f"Invalid character detected: {repr(expression[i])}")
            tokens.append(expression[i])
            i += 1

    return tokens


def sanitizer_validator(expression):
    try:
        expression = expression.replace('×', '*')
        expression = expression.replace('−', '-')
        expression = expression.replace('^', '**')
        expression = expression.replace('=', '')
        expression = expression.replace('°', '')  # Handling degree symbol
        return expression
    except Exception as e:
        print(f"ERROR in function: sanitizer_validator(expression) = {str(e)}")


def evaluate_recipe(recipe):
    stack = []
    for operation, value in recipe:
        # # inspection
        # print(f"evaluate_recipe(), Processing operation: {operation} with value -> {value}")

        # print(f"Processing {operation}, Value: {value}")
        # # [Existing logic for handling different operations]
        # print(f"Current stack: {stack}")

        # Check and convert 'pi' and 'e' to their numerical values
        if value == 'pi':
            stack.append(pi)  # Push the value of pi onto the stack
        elif value == 'e':
            stack.append(e)  # Push the value of e onto the stack

        elif operation == "PUSH":
            stack.append(value)
        elif operation == "OPERATOR":
            if value == 'pi':
                value = pi
            elif value == 'e':
                value = e
            elif value == 'u-':  # Unary minus
                if len(stack) < 1:
                    raise ValueError("Not enough operands for unary minus")
                operand = stack.pop()
                result = -operand
            elif value == "-":
                if len(stack) < 2:
                    raise ValueError("Not enough operands for subtraction")
                operand2 = stack.pop()
                operand1 = stack.pop()
                result = operand1 - operand2
            elif value in ["sin", "cos", "tan"]:
                operand = radians(stack.pop())  # Convert to radians
                if value == "sin":
                    result = sin(operand)
                elif value == "cos":
                    result = cos(operand)
                elif value == "tan":
                    result = tan(operand)
            elif value == 'abs':
                operand = stack.pop()
                result = abs(operand)
            elif value == 'sqrt':
                operand = stack.pop()
                if operand < 0:
                    raise ValueError("Cannot calculate the square root of a negative number")
                result = sqrt(operand)
                # stack.append(result)
            else:
                operand2 = stack.pop()
                operand1 = stack.pop()
                if value == "+":
                    result = operand1 + operand2
                elif value == "-":
                    result = operand1 - operand2
                elif value == "**":
                    result = operand1 ** operand2
                elif value == "*":
                    result = operand1 * operand2
                elif value == "/":
                    if operand2 == 0:
                        raise ValueError("Division by zero error")
                    result = operand1 / operand2

            stack.append(result)  # Push the result back onto the stack

    # inspection
    # print(f"Final stack state: {stack}")

    return stack[0]


# ALU helper
def rpn_calculator(raw_expression):
    try:
        # print(f"original expression - > {raw_expression}")
        expression = sanitizer_validator(raw_expression)
        # print(f"sanitized expression - > {expression}")

        try:
            tokens = tokenize(expression)
            rpn = shunting_yard(tokens)
            recipe = []
            for token in rpn:
                if isinstance(token, (int, float)):
                    recipe.append(("PUSH", token))
                else:
                    recipe.append(("OPERATOR", token))
        except ValueError as e:
            print(f"Error in tokenization or shunting yard: {e}")
            return (raw_expression, [], "Error: " + str(e))

        final_result = evaluate_recipe(recipe)
        # print(f"final_result - > {final_result}")

        result_tuple = (raw_expression, recipe, final_result)
        print(f"result_tuple -> {result_tuple}")

        return result_tuple

    except Exception as e:
        print(f"ERROR in function: rpn_calculator(raw_expression), for {raw_expression}  = {str(e)}")
        return (raw_expression, [], "Error: " + str(e))



# def rpn_calculator(raw_expression):

#     try:

#         """
#         return tuple
#         ("original expression string", [list, of, steps,], solution_value)
#         """

#         print(f"original expression - > {raw_expression}")
#         expression = sanitizer_validator(raw_expression)
#         print(f"sanitized expression - > {expression}")

#         recipe = to_rpn(expression)
#         print(f"recipe - > {recipe}")

#         final_result = evaluate_recipe(recipe)
#         print(f"final_result - > {final_result}")

#         # make tuple
#         result_tuple = (raw_expression, recipe, final_result)
#         print(f"result_tuple -> {result_tuple}")

#         return result_tuple

#     except Exception as e:
#         print(f"ERROR in function: rpn_calculator(raw_expression), for {raw_expression}  = {str(e)}" )


# ALU helper
def alu_list_rpn_calculator(raw_expression_list):
    """
    example use:
    alu_list_rpn_calculator(["1+1"])
    """
    try:
        # print(f" alu_list_rpn_calculator() incoming raw_expression_list -> {raw_expression_list}")

        report_list = []
        for i in raw_expression_list:
            #  print(f" for i in raw_expression_list: i -> {i}")

            result = rpn_calculator(i)
            # print(f" result -> {result}")

            report_list.append( result )

        return report_list

    except Exception as e:
        print(f"ERROR in function: alu_list_rpn_calculator(raw_expression_list), for expression {i}  = {str(e)}" )


# ALU helper function
def alu_find_equivalent_expressions(expressions):
    """
    Step 1: make dicionary
    Step 2: report as tuples

    Starting with this format:
    ("original expression string", [list, of, steps,], solution_value)
    find equivilance between expressions in the tuple list,
    returning perhaps a list of tuples with each tuple representing
    equivilant expressions, and inside being first a list
    of those expressions and then the value they all equal
    e.g. [([exp1,exp3],5), ([exp2,exp4], .5)]
    """
    try:
        # Create a dictionary to group expressions by their solution values
        equivalence_dict = {}
        for expression, _, value in expressions:
            if value not in equivalence_dict:
                equivalence_dict[value] = []
            equivalence_dict[value].append(expression)

        # Create the final list of tuples containing equivalent expressions and their values
        result = [(expressions, value) for value, expressions in equivalence_dict.items() if len(expressions) > 1]

        return result

    except Exception as e:
        print(f"ERROR in function: alu_find_equivalent_expressions(expressions)  = {str(e)}" )




###########
# Tests 1
###########


def mathlist_run(math_list_string):
    # print("math_list_string", math_list_string)
    math_list = math_list_string.split(',')
    for i in math_list:
        # print( alu_list_rpn_calculator([i]))
        alu_list_rpn_calculator([i])


i = "(7 - 3) - (5 - 10)"
# # i = "(7) - (10)"
alu_list_rpn_calculator([i])

alu_list_rpn_calculator(["1+1"])

alu_list_rpn_calculator(["sin(30°)"])

alu_list_rpn_calculator(["sin 30"])

alu_list_rpn_calculator(["abs(-3)"])

math_list_string = """3 + 4 * 2 / ( 1 - 5 ),-5 - (-6),
−4×(−9),−4*(−9),
5 + (-3), 10 + (-7), -4 + 11, -2 + 9,
(-5) + (-3), (-10) - (-7), (-15) + (-8), (-4) - 11,
5 - (-3), 10 - (-7), 15 - (-8), 4 - (-11),
5 * (-3), 10 * (-7), -4 * 11, -2 * 9,
5 / (-3), 10 / (-7), -4 / 11, -2 / 9,
abs(5), abs(-3), abs(10), abs(-7),
5**2, (-3)**2, 10**2, (-7)**2,
5**3, 3**4, 2**7, 6**2,
0.5**3, 0.3**4, 0.2**7, 0.6**2,
2**3, 3**4, 4**2, sin(30°) ,cos(30°) , tan(30°)"""

mathlist_run(math_list_string)

##############
# Test Set 2
##############

# Basic Arithmetic Operations: Test all basic arithmetic operations (addition, subtraction, multiplication, division) with positive, negative, and zero values.
math_list = ["1 + 2", "-3 - 4", "5 * -6", "7 / -8", "0 * 3", "-5 + 0"]
alu_list_rpn_calculator(math_list)

# alu_list_rpn_calculator(["5 * -6"])

# Unary Operations: Check the handling of unary minus in various contexts.
math_list = ["-(3 + 4)", "-3 + 4", "3 + -4", "--5", "-(-3 - -4)"]
alu_list_rpn_calculator(math_list)

alu_list_rpn_calculator(["-(-3 - -4)"])


# Floating Point Numbers: Test with floating-point numbers to ensure accurate handling of decimal values.
math_list = ["3.5 + 2.1", "-4.7 * 0.5"]
alu_list_rpn_calculator(math_list)

# Parentheses and Order of Operations: Test expressions with nested parentheses and mixed operations to check if the order of operations is respected.
math_list = ["(2 + 3) * 4", "2 * (3 + 4)", "(5 - (6 / 2)) * 3"]
alu_list_rpn_calculator(math_list)

# Functions and Advanced Operations: Include trigonometric functions, exponentiation, square roots, and absolute values.
math_list = ["sin(30)", "cos(-45)", "tan(60)", "2 ** 3", "sqrt(16)", "abs(-9)"]
alu_list_rpn_calculator(math_list)

# Edge Cases: Test cases that often result in errors like division by zero, very large and small numbers, and invalid inputs.
math_list = ["1 / 0", "99999999 * 9999999", "1 / 0.0000001", "sin(90) / cos(90)", "sqrt(-1)"]
alu_list_rpn_calculator(math_list)

# Complex Expressions: Combine multiple operations in a single expression to test complex parsing.
math_list = ["3 + 4 * 2 / (1 - 5)", "5 - -3 + 2 * 4"]
alu_list_rpn_calculator(math_list)

# Whitespace and Formatting Variations: Ensure the parser can handle expressions with varying whitespace and formatting.
math_list = [" 3+5 ", "\n4*5", " 7\t- 2 "]
alu_list_rpn_calculator(math_list)

# Special Mathematical Constants and Variables: If your calculator supports it, include tests with special constants like π, e, etc.
math_list = ["2 * pi", "e ** 2"]
alu_list_rpn_calculator(math_list)

# Invalid Expressions: Test with invalid or malformed expressions to ensure appropriate error handling.
math_list = ["2 ++ 2", "sin(30 + 60", "2 */ 3"]
alu_list_rpn_calculator(math_list)



result_tuple -> ('(7 - 3) - (5 - 10)', [('PUSH', 7.0), ('PUSH', 3.0), ('OPERATOR', '-'), ('PUSH', 5.0), ('PUSH', 10.0), ('OPERATOR', '-'), ('OPERATOR', '-')], 9.0)
result_tuple -> ('1+1', [('PUSH', 1.0), ('PUSH', 1.0), ('OPERATOR', '+')], 2.0)
result_tuple -> ('sin(30°)', [('PUSH', 30.0), ('OPERATOR', 'sin')], 0.49999999999999994)
result_tuple -> ('sin 30', [('PUSH', 30.0), ('OPERATOR', 'sin')], 0.49999999999999994)
result_tuple -> ('abs(-3)', [('PUSH', 3.0), ('OPERATOR', 'u-'), ('OPERATOR', 'abs')], 3.0)
result_tuple -> ('3 + 4 * 2 / ( 1 - 5 )', [('PUSH', 3.0), ('PUSH', 4.0), ('PUSH', 2.0), ('OPERATOR', '*'), ('PUSH', 1.0), ('PUSH', 5.0), ('OPERATOR', '-'), ('OPERATOR', '/'), ('OPERATOR', '+')], 1.0)
result_tuple -> ('-5 - (-6)', [('PUSH', 5.0), ('OPERATOR', 'u-'), ('PUSH', 6.0), ('OPERATOR', 'u-'), ('OPERATOR', '-')], 1.0)
result_tuple -> ('\n−4×(−9)', [('PUSH', 4.0), ('OPERATOR', 'u-'), ('PUSH', 9.0), ('OPERATOR', 'u-'), ('OPERATOR', '*')], 36.0)
result_tuple -> ('−4*(−9)', [('PUSH'

[('2 ++ 2', [], 'Error: Invalid consecutive operators: ++ at position 1'),
 ('sin(30 + 60', [], 'Error: Unbalanced parentheses in expression'),
 ('2 */ 3', [], 'Error: Invalid consecutive operators: */ at position 1')]

In [2]:
math_list = """3 + 4 * 2 / ( 1 - 5 ),-5 - (-6),
−4×(−9),−4*(−9),
5 + (-3), 10 + (-7), -4 + 11, -2 + 9,
(-5) + (-3), (-10) - (-7), (-15) + (-8), (-4) - 11,
5 - (-3), 10 - (-7), 15 - (-8), 4 - (-11),
5 * (-3), 10 * (-7), -4 * 11, -2 * 9,
5 / (-3), 10 / (-7), -4 / 11, -2 / 9,
abs(5), abs(-3), abs(10), abs(-7),
5**2, (-3)**2, 10**2, (-7)**2,
5**3, 3**4, 2**7, 6**2,
0.5**3, 0.3**4, 0.2**7, 0.6**2,
2**3, 3**4, 4**2, sin(30°) ,cos(30°) , tan(30°)"""

math_list = math_list.split(',')

for i in math_list:
  print( alu_list_rpn_calculator([i]) )

result_tuple -> ('3 + 4 * 2 / ( 1 - 5 )', [('PUSH', 3.0), ('PUSH', 4.0), ('PUSH', 2.0), ('OPERATOR', '*'), ('PUSH', 1.0), ('PUSH', 5.0), ('OPERATOR', '-'), ('OPERATOR', '/'), ('OPERATOR', '+')], 1.0)
[('3 + 4 * 2 / ( 1 - 5 )', [('PUSH', 3.0), ('PUSH', 4.0), ('PUSH', 2.0), ('OPERATOR', '*'), ('PUSH', 1.0), ('PUSH', 5.0), ('OPERATOR', '-'), ('OPERATOR', '/'), ('OPERATOR', '+')], 1.0)]
result_tuple -> ('-5 - (-6)', [('PUSH', 5.0), ('OPERATOR', 'u-'), ('PUSH', 6.0), ('OPERATOR', 'u-'), ('OPERATOR', '-')], 1.0)
[('-5 - (-6)', [('PUSH', 5.0), ('OPERATOR', 'u-'), ('PUSH', 6.0), ('OPERATOR', 'u-'), ('OPERATOR', '-')], 1.0)]
result_tuple -> ('\n−4×(−9)', [('PUSH', 4.0), ('OPERATOR', 'u-'), ('PUSH', 9.0), ('OPERATOR', 'u-'), ('OPERATOR', '*')], 36.0)
[('\n−4×(−9)', [('PUSH', 4.0), ('OPERATOR', 'u-'), ('PUSH', 9.0), ('OPERATOR', 'u-'), ('OPERATOR', '*')], 36.0)]
result_tuple -> ('−4*(−9)', [('PUSH', 4.0), ('OPERATOR', 'u-'), ('PUSH', 9.0), ('OPERATOR', 'u-'), ('OPERATOR', '*')], 36.0)
[('−4*(−9)'